# Importing Libraries

In [1]:
# Importing pre-trained model
from keras.applications import ResNet50

# Importing important layers of Keras
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.callbacks import EarlyStopping

# Importing data functions
from data_library.data_processing import black_white, crop, delete_face_images, encode, to_cat

# Importing preprocessing functions
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical


# Importing the pre-trained model

In [2]:
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator

# Saving the model in a variable
resnet = ResNet50(weights='imagenet', include_top=False)


94765736/94765736 [==============================] - 16s 0us/step


# 10 Classes from Real Madrid (cropped_faces)

In [3]:
faces_folder = "../data_processing/raw_data/faces/"
cropped_faces_directory = "../data_processing/raw_data/cropped_faces"


In [4]:
y, X = encode(cropped_faces_directory)
y_cat = to_cat(y)

X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.2)


## Model 1

In [5]:
# Add a new classification head
layer = resnet.output
layer = GlobalAveragePooling2D()(layer)
predictions = Dense(y_train.shape[1], activation='softmax')(layer)

model = Model(inputs=resnet.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Callbacks
es = EarlyStopping(patience=3,
                   restore_best_weights=True)

# Train the model
model.fit(X_train,
          y_train,
          epochs=10,
          batch_size=16,
          validation_split=0.2,
          callbacks=[es]
         )


Epoch 1/10


2023-11-25 12:40:14.117352: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


10/10 [==============================] - 5s 232ms/step - loss: 2.2123 - accuracy: 0.3782 - val_loss: 15.1125 - val_accuracy: 0.2250
Epoch 2/10
10/10 [==============================] - 2s 179ms/step - loss: 1.1137 - accuracy: 0.7115 - val_loss: 335.9442 - val_accuracy: 0.1250
Epoch 3/10
10/10 [==============================] - 2s 180ms/step - loss: 0.9770 - accuracy: 0.8782 - val_loss: 5980.7432 - val_accuracy: 0.1250
Epoch 4/10
10/10 [==============================] - 2s 183ms/step - loss: 0.5295 - accuracy: 0.8910 - val_loss: 32339.6777 - val_accuracy: 0.1250


In [6]:
model.evaluate(X_test, y_test)


2/2 [==============================] - 0s 67ms/step - loss: 16.7883 - accuracy: 0.1633


[16.788280487060547, 0.16326530277729034]

## Model 2

In [7]:
# Define the input shape for your images
input_shape = X_train.shape[1:]

# Define a Sequential model or Functional API model
input_layer = Input(shape=input_shape)

# Add augmentation layers directly within the model
augmented = Conv2D(32, (3, 3), activation='relu')(input_layer)
augmented = MaxPooling2D((2, 2))(augmented)
# Add more layers or augmentation techniques as needed

# Rest of your model architecture
# Example:
augmented = Flatten()(augmented)
augmented = Dense(128, activation='relu')(augmented)
output_layer = Dense(y_train.shape[1], activation='softmax')(augmented)


model_2 = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model_2.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Callbacks
es = EarlyStopping(patience=3,
                   restore_best_weights=True)

datagen = ImageDataGenerator(
    rotation_range=40,       # Rotate images by a wider range of degrees
    width_shift_range=0.3,   # Shift images horizontally by a larger fraction of total width
    height_shift_range=0.3,  # Shift images vertically by a larger fraction of total height
    shear_range=0.3,         # Apply a larger shear-based transformation
    zoom_range=0.3,          # Zoom in or out of images more aggressively
    horizontal_flip=True,    # Flip images horizontally
    vertical_flip=True,      # Flip images vertically
    fill_mode='nearest'      # Fill points outside the boundaries using the nearest available value
)

# Fit the ImageDataGenerator to your data
datagen.fit(X_train)

# Example of using the ImageDataGenerator during model training
model_2.fit(datagen.flow(X_train, y_train, batch_size=32), epochs=100)


Epoch 1/100
7/7 [==============================] - 0s 16ms/step - loss: 980.6898 - accuracy: 0.0816
Epoch 2/100
7/7 [==============================] - 0s 17ms/step - loss: 1007.4955 - accuracy: 0.0867
Epoch 3/100
7/7 [==============================] - 0s 15ms/step - loss: 304.1216 - accuracy: 0.1122
Epoch 4/100
7/7 [==============================] - 0s 14ms/step - loss: 106.2029 - accuracy: 0.0918
Epoch 5/100
7/7 [==============================] - 0s 15ms/step - loss: 25.7008 - accuracy: 0.1020
Epoch 6/100
7/7 [==============================] - 0s 14ms/step - loss: 9.3852 - accuracy: 0.1684
Epoch 7/100
7/7 [==============================] - 0s 17ms/step - loss: 5.6860 - accuracy: 0.1224
Epoch 8/100
7/7 [==============================] - 0s 18ms/step - loss: 4.2978 - accuracy: 0.1224
Epoch 9/100
7/7 [==============================] - 0s 17ms/step - loss: 3.8512 - accuracy: 0.1276
Epoch 10/100
7/7 [==============================] - 0s 16ms/step - loss: 3.8347 - accuracy: 0.0918
Epoch 11/

In [8]:
model_2.evaluate(X_test, y_test)


2/2 [==============================] - 0s 4ms/step - loss: 2.8762 - accuracy: 0.0816


[2.876171827316284, 0.08163265138864517]

## Model 3

In [9]:
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define the input shape for your images
input_shape = X_train.shape[1:]

# Define a Sequential model or Functional API model
input_layer = Input(shape=input_shape)

# Add convolutional layers with pooling
conv1 = Conv2D(32, (3, 3), activation='relu')(input_layer)
pool1 = MaxPooling2D((2, 2))(conv1)

conv2 = Conv2D(64, (3, 3), activation='relu')(pool1)
pool2 = MaxPooling2D((2, 2))(conv2)

# Flatten the output before feeding into dense layers
flatten = Flatten()(pool2)

# Add dense layers with dropout for regularization
dense1 = Dense(128, activation='relu')(flatten)
dropout1 = Dropout(0.5)(dense1)  # Dropout layer for regularization

dense2 = Dense(64, activation='relu')(dropout1)
dropout2 = Dropout(0.5)(dense2)  # Dropout layer for regularization

# Output layer with softmax activation for multi-class classification
output_layer = Dense(y_train.shape[1], activation='softmax')(dropout2)

# Create the model
model_3 = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model_3.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Create an ImageDataGenerator with multiple augmentation techniques
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.5, 1.5],  # Vary brightness
    channel_shift_range=50,        # Vary channel shifts
    fill_mode='nearest'
)

# Fit the ImageDataGenerator to your data
datagen.fit(X_train)

# Example of using the ImageDataGenerator during model training
model_3.fit(datagen.flow(X_train, y_train, batch_size=32), epochs=100)


Epoch 1/100
7/7 [==============================] - 0s 26ms/step - loss: 123.4912 - accuracy: 0.1173
Epoch 2/100
7/7 [==============================] - 0s 21ms/step - loss: 5.1083 - accuracy: 0.1071
Epoch 3/100
7/7 [==============================] - 0s 26ms/step - loss: 2.4844 - accuracy: 0.0816
Epoch 4/100
7/7 [==============================] - 0s 22ms/step - loss: 2.3987 - accuracy: 0.0765
Epoch 5/100
7/7 [==============================] - 0s 20ms/step - loss: 2.3471 - accuracy: 0.0918
Epoch 6/100
7/7 [==============================] - 0s 23ms/step - loss: 2.3321 - accuracy: 0.1173
Epoch 7/100
7/7 [==============================] - 0s 21ms/step - loss: 2.2930 - accuracy: 0.1224
Epoch 8/100
7/7 [==============================] - 0s 23ms/step - loss: 2.3089 - accuracy: 0.1020
Epoch 9/100
7/7 [==============================] - 0s 22ms/step - loss: 2.2994 - accuracy: 0.1224
Epoch 10/100
7/7 [==============================] - 0s 21ms/step - loss: 2.2909 - accuracy: 0.1020
Epoch 11/100
7/7 

In [10]:
model_3.evaluate(X_test, y_test)


2/2 [==============================] - 0s 5ms/step - loss: 2.3019 - accuracy: 0.1020


[2.3019301891326904, 0.10204081982374191]

# Video - 10 Classes (video_cropped_faces_0)

In [11]:
video_cropped_faces_directory = "../data_processing/raw_data/video_cropped_faces_0"


In [12]:
y, X = encode(video_cropped_faces_directory)
y_cat = to_cat(y)
X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.2)


In [13]:
y_train.shape


(63, 11)

## Model 1

In [14]:
# Add a new classification head
layer = resnet.output
layer = GlobalAveragePooling2D()(layer)
predictions = Dense(y_train.shape[1], activation='softmax')(layer)

model_1 = Model(inputs=resnet.input, outputs=predictions)

# Compile the model
model_1.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Callbacks
es = EarlyStopping(patience=3,
                   restore_best_weights=True)

# Train the model
model_1.fit(X_train,
          y_train,
          epochs=10,
          batch_size=16,
          validation_split=0.2,
          callbacks=[es]
         )


Epoch 1/10
4/4 [==============================] - 4s 304ms/step - loss: 2.5330 - accuracy: 0.3800 - val_loss: 22.5859 - val_accuracy: 0.2308
Epoch 2/10
4/4 [==============================] - 1s 149ms/step - loss: 2.0179 - accuracy: 0.5800 - val_loss: 131.7289 - val_accuracy: 0.2308
Epoch 3/10
4/4 [==============================] - 1s 148ms/step - loss: 0.5238 - accuracy: 0.8800 - val_loss: 334.8791 - val_accuracy: 0.0769
Epoch 4/10
4/4 [==============================] - 1s 158ms/step - loss: 0.4564 - accuracy: 0.8400 - val_loss: 1184.3062 - val_accuracy: 0.0769


In [15]:
model_1.evaluate(X_test, y_test)


1/1 [==============================] - 0s 66ms/step - loss: 21.5645 - accuracy: 0.1875


[21.564544677734375, 0.1875]

## Model 2

In [16]:
# Define the input shape for your images
input_shape = X_train.shape[1:]

# Define a Sequential model or Functional API model
input_layer = Input(shape=input_shape)

# Add augmentation layers directly within the model
augmented = Conv2D(32, (3, 3), activation='relu')(input_layer)
augmented = MaxPooling2D((2, 2))(augmented)
# Add more layers or augmentation techniques as needed

# Rest of your model architecture
# Example:
augmented = Flatten()(augmented)
augmented = Dense(128, activation='relu')(augmented)
output_layer = Dense(y_train.shape[1], activation='softmax')(augmented)


model_2 = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model_2.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Callbacks
es = EarlyStopping(patience=3,
                   restore_best_weights=True)

datagen = ImageDataGenerator(
    rotation_range=40,       # Rotate images by a wider range of degrees
    width_shift_range=0.3,   # Shift images horizontally by a larger fraction of total width
    height_shift_range=0.3,  # Shift images vertically by a larger fraction of total height
    shear_range=0.3,         # Apply a larger shear-based transformation
    zoom_range=0.3,          # Zoom in or out of images more aggressively
    horizontal_flip=True,    # Flip images horizontally
    vertical_flip=True,      # Flip images vertically
    fill_mode='nearest'      # Fill points outside the boundaries using the nearest available value
)

# Fit the ImageDataGenerator to your data
datagen.fit(X_train)

# Example of using the ImageDataGenerator during model training
model_2.fit(datagen.flow(X_train, y_train, batch_size=32), epochs=100)


Epoch 1/100
2/2 [==============================] - 0s 19ms/step - loss: 348.5375 - accuracy: 0.1587
Epoch 2/100
2/2 [==============================] - 0s 16ms/step - loss: 761.2393 - accuracy: 0.1111
Epoch 3/100
2/2 [==============================] - 0s 15ms/step - loss: 637.9498 - accuracy: 0.0952
Epoch 4/100
2/2 [==============================] - 0s 16ms/step - loss: 373.7281 - accuracy: 0.2063
Epoch 5/100
2/2 [==============================] - 0s 16ms/step - loss: 233.9421 - accuracy: 0.1429
Epoch 6/100
2/2 [==============================] - 0s 15ms/step - loss: 188.9757 - accuracy: 0.1429
Epoch 7/100
2/2 [==============================] - 0s 18ms/step - loss: 109.6864 - accuracy: 0.1111
Epoch 8/100
2/2 [==============================] - 0s 14ms/step - loss: 65.4010 - accuracy: 0.1429
Epoch 9/100
2/2 [==============================] - 0s 15ms/step - loss: 40.9583 - accuracy: 0.2381
Epoch 10/100
2/2 [==============================] - 0s 17ms/step - loss: 45.7670 - accuracy: 0.1905
Ep

In [17]:
model_2.evaluate(X_test, y_test)


1/1 [==============================] - 0s 66ms/step - loss: 2.2410 - accuracy: 0.4375


[2.241020679473877, 0.4375]

## Model 3

In [18]:
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define the input shape for your images
input_shape = X_train.shape[1:]

# Define a Sequential model or Functional API model
input_layer = Input(shape=input_shape)

# Add convolutional layers with pooling
conv1 = Conv2D(32, (3, 3), activation='relu')(input_layer)
pool1 = MaxPooling2D((2, 2))(conv1)

conv2 = Conv2D(64, (3, 3), activation='relu')(pool1)
pool2 = MaxPooling2D((2, 2))(conv2)

# Flatten the output before feeding into dense layers
flatten = Flatten()(pool2)

# Add dense layers with dropout for regularization
dense1 = Dense(128, activation='relu')(flatten)
dropout1 = Dropout(0.5)(dense1)  # Dropout layer for regularization

dense2 = Dense(64, activation='relu')(dropout1)
dropout2 = Dropout(0.5)(dense2)  # Dropout layer for regularization

# Output layer with softmax activation for multi-class classification
output_layer = Dense(y_train.shape[1], activation='softmax')(dropout2)

# Create the model
model_3 = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model_3.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Create an ImageDataGenerator with multiple augmentation techniques
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.5, 1.5],  # Vary brightness
    channel_shift_range=50,        # Vary channel shifts
    fill_mode='nearest'
)

# Fit the ImageDataGenerator to your data
datagen.fit(X_train)

# Example of using the ImageDataGenerator during model training
model_3.fit(datagen.flow(X_train, y_train, batch_size=32), epochs=100)


Epoch 1/100
2/2 [==============================] - 0s 21ms/step - loss: 49.0447 - accuracy: 0.0952
Epoch 2/100
2/2 [==============================] - 0s 21ms/step - loss: 68.1441 - accuracy: 0.0635
Epoch 3/100
2/2 [==============================] - 0s 22ms/step - loss: 26.8890 - accuracy: 0.1111
Epoch 4/100
2/2 [==============================] - 0s 19ms/step - loss: 14.8381 - accuracy: 0.0794
Epoch 5/100
2/2 [==============================] - 0s 22ms/step - loss: 5.9774 - accuracy: 0.1429
Epoch 6/100
2/2 [==============================] - 0s 31ms/step - loss: 3.0450 - accuracy: 0.1746
Epoch 7/100
2/2 [==============================] - 0s 22ms/step - loss: 2.7462 - accuracy: 0.0635
Epoch 8/100
2/2 [==============================] - 0s 26ms/step - loss: 2.6509 - accuracy: 0.1111
Epoch 9/100
2/2 [==============================] - 0s 27ms/step - loss: 2.5149 - accuracy: 0.1429
Epoch 10/100
2/2 [==============================] - 0s 21ms/step - loss: 2.4391 - accuracy: 0.1270
Epoch 11/100
2/

In [19]:
model_3.evaluate(X_test, y_test)


1/1 [==============================] - 0s 78ms/step - loss: 1.8065 - accuracy: 0.4375


[1.8064758777618408, 0.4375]

## Model 4

In [20]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

model_4 = Sequential()
model_4.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model_4.add(MaxPooling2D((2, 2)))
model_4.add(Conv2D(64, (3, 3), activation='relu'))
model_4.add(MaxPooling2D((2, 2)))
model_4.add(Conv2D(128, (3, 3), activation='relu'))
model_4.add(MaxPooling2D((2, 2)))
model_4.add(Flatten())
model_4.add(Dense(256, activation='relu', kernel_regularizer='l2'))  # Adding L2 regularization
model_4.add(Dropout(0.5))  # Adding Dropout layer
model_4.add(Dense(128, activation='relu', kernel_regularizer='l2'))  # Adding L2 regularization
model_4.add(Dense(y_train.shape[1], activation='softmax'))

# Using Adam optimizer with a reduced learning rate
optimizer = Adam(lr=0.0001)
model_4.compile(optimizer="adam",
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Adding more callbacks, e.g., ReduceLROnPlateau
callbacks = [EarlyStopping(patience=5, restore_best_weights=True),
             ReduceLROnPlateau(factor=0.1, patience=3)]

# Fit the model with increased complexity and callbacks
history = model_4.fit(datagen.flow(X_train, y_train, batch_size=16),
                    epochs=100,
                    callbacks=callbacks,
                    validation_data=(X_test, y_test))


Epoch 1/100


/Users/yassinebouaine/.pyenv/versions/3.10.6/envs/dynamic-players-insights/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


4/4 [==============================] - 0s 49ms/step - loss: 31.8144 - accuracy: 0.0794 - val_loss: 13.5082 - val_accuracy: 0.0625 - lr: 0.0010
Epoch 2/100
4/4 [==============================] - 0s 20ms/step - loss: 12.1870 - accuracy: 0.1270 - val_loss: 7.7278 - val_accuracy: 0.2500 - lr: 0.0010
Epoch 3/100
4/4 [==============================] - 0s 19ms/step - loss: 7.7952 - accuracy: 0.0952 - val_loss: 7.3807 - val_accuracy: 0.1250 - lr: 0.0010
Epoch 4/100
4/4 [==============================] - 0s 19ms/step - loss: 7.1831 - accuracy: 0.1270 - val_loss: 7.0096 - val_accuracy: 0.0625 - lr: 0.0010
Epoch 5/100
4/4 [==============================] - 0s 19ms/step - loss: 6.8917 - accuracy: 0.2063 - val_loss: 6.6498 - val_accuracy: 0.4375 - lr: 0.0010
Epoch 6/100
4/4 [==============================] - 0s 21ms/step - loss: 6.6689 - accuracy: 0.1429 - val_loss: 6.6314 - val_accuracy: 0.0625 - lr: 0.0010
Epoch 7/100
4/4 [==============================] - 0s 19ms/step - loss: 6.5239 - accuracy: 

In [21]:
model_4.evaluate(X_test, y_test)


1/1 [==============================] - 0s 15ms/step - loss: 4.7072 - accuracy: 0.6250


[4.707248210906982, 0.625]

## Model 5

In [22]:
from keras.layers import BatchNormalization, LeakyReLU

model_5 = Sequential()
model_5.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model_5.add(MaxPooling2D((2, 2)))
model_5.add(BatchNormalization())  # Add BatchNormalization after each Conv layer
model_5.add(Conv2D(64, (3, 3), activation='relu'))
model_5.add(MaxPooling2D((2, 2)))
model_5.add(BatchNormalization())
model_5.add(Conv2D(128, (3, 3), activation='relu'))
model_5.add(MaxPooling2D((2, 2)))
model_5.add(BatchNormalization())
model_5.add(Flatten())
model_5.add(Dense(256))
model_5.add(LeakyReLU(alpha=0.1))  # LeakyReLU activation
model_5.add(Dropout(0.5))
model_5.add(Dense(128))
model_5.add(LeakyReLU(alpha=0.1))
model_5.add(Dense(y_train.shape[1], activation='softmax'))

# Compile the model
model_5.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Training the model with adjusted architecture
es = EarlyStopping(patience=10, restore_best_weights=True)
history = model_5.fit(datagen.flow(X_train, y_train, batch_size=64),
                    epochs=150,  # Train for more epochs
                    callbacks=es,
                    validation_data=(X_test, y_test))


Epoch 1/150
1/1 [==============================] - 1s 664ms/step - loss: 2.9954 - accuracy: 0.0794 - val_loss: 14.4994 - val_accuracy: 0.0000e+00
Epoch 2/150
1/1 [==============================] - 0s 106ms/step - loss: 2.9037 - accuracy: 0.2063 - val_loss: 13.3427 - val_accuracy: 0.0625
Epoch 3/150
1/1 [==============================] - 0s 106ms/step - loss: 3.0453 - accuracy: 0.2222 - val_loss: 6.3568 - val_accuracy: 0.0625
Epoch 4/150
1/1 [==============================] - 0s 103ms/step - loss: 2.6630 - accuracy: 0.2698 - val_loss: 7.0638 - val_accuracy: 0.0625
Epoch 5/150
1/1 [==============================] - 0s 104ms/step - loss: 2.9225 - accuracy: 0.2381 - val_loss: 7.6025 - val_accuracy: 0.0625
Epoch 6/150
1/1 [==============================] - 0s 101ms/step - loss: 2.5968 - accuracy: 0.2381 - val_loss: 6.5506 - val_accuracy: 0.0625
Epoch 7/150
1/1 [==============================] - 0s 103ms/step - loss: 2.1709 - accuracy: 0.3492 - val_loss: 6.0982 - val_accuracy: 0.0625
Epoch 8

In [23]:
model_5.evaluate(X_test, y_test)


1/1 [==============================] - 0s 17ms/step - loss: 2.1740 - accuracy: 0.3125


[2.1739940643310547, 0.3125]

# Video + Getty - 10 Classes (video_cropped_faces_1)

In [24]:
cropped_faces_directory = "../data_processing/raw_data/video_cropped_faces_1"


In [25]:
y, X = encode(cropped_faces_directory)
y_cat = to_cat(y)


In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.2)


## Model 1

In [27]:
# Add a new classification head
layer = resnet.output
layer = GlobalAveragePooling2D()(layer)
predictions = Dense(y_train.shape[1], activation='softmax')(layer)

model_1 = Model(inputs=resnet.input, outputs=predictions)

# Compile the model
model_1.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Callbacks
es = EarlyStopping(patience=3,
                   restore_best_weights=True)

# Train the model
model_1.fit(X_train,
          y_train,
          epochs=10,
          batch_size=16,
          validation_split=0.2,
          callbacks=[es]
         )


Epoch 1/10
13/13 [==============================] - 6s 244ms/step - loss: 2.9072 - accuracy: 0.4118 - val_loss: 39067.3164 - val_accuracy: 0.1154
Epoch 2/10
13/13 [==============================] - 3s 202ms/step - loss: 2.5454 - accuracy: 0.5049 - val_loss: 12423.9062 - val_accuracy: 0.0962
Epoch 3/10
13/13 [==============================] - 3s 206ms/step - loss: 1.9935 - accuracy: 0.5098 - val_loss: 60125.4531 - val_accuracy: 0.0962
Epoch 4/10
13/13 [==============================] - 3s 211ms/step - loss: 1.9865 - accuracy: 0.5686 - val_loss: 89333.4844 - val_accuracy: 0.0769
Epoch 5/10
13/13 [==============================] - 3s 224ms/step - loss: 1.3524 - accuracy: 0.6373 - val_loss: 2755.1169 - val_accuracy: 0.0385
Epoch 6/10
13/13 [==============================] - 3s 232ms/step - loss: 0.8082 - accuracy: 0.7549 - val_loss: 1261.1185 - val_accuracy: 0.0385
Epoch 7/10
13/13 [==============================] - 3s 215ms/step - loss: 0.6787 - accuracy: 0.8088 - val_loss: 672.5253 - val

In [28]:
model_1.evaluate(X_test, y_test)


2/2 [==============================] - 0s 78ms/step - loss: 23.1061 - accuracy: 0.1875


[23.10612678527832, 0.1875]

## Model 2

In [29]:
# Define the input shape for your images
input_shape = X_train.shape[1:]

# Define a Sequential model or Functional API model
input_layer = Input(shape=input_shape)

# Add augmentation layers directly within the model
augmented = Conv2D(32, (3, 3), activation='relu')(input_layer)
augmented = MaxPooling2D((2, 2))(augmented)
# Add more layers or augmentation techniques as needed

# Rest of your model architecture
# Example:
augmented = Flatten()(augmented)
augmented = Dense(128, activation='relu')(augmented)
output_layer = Dense(y_train.shape[1], activation='softmax')(augmented)


model_2 = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model_2.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Callbacks
es = EarlyStopping(patience=3,
                   restore_best_weights=True)

datagen = ImageDataGenerator(
    rotation_range=40,       # Rotate images by a wider range of degrees
    width_shift_range=0.3,   # Shift images horizontally by a larger fraction of total width
    height_shift_range=0.3,  # Shift images vertically by a larger fraction of total height
    shear_range=0.3,         # Apply a larger shear-based transformation
    zoom_range=0.3,          # Zoom in or out of images more aggressively
    horizontal_flip=True,    # Flip images horizontally
    vertical_flip=True,      # Flip images vertically
    fill_mode='nearest'      # Fill points outside the boundaries using the nearest available value
)

# Fit the ImageDataGenerator to your data
datagen.fit(X_train)

# Example of using the ImageDataGenerator during model training
model_2.fit(datagen.flow(X_train, y_train, batch_size=32), epochs=100)


Epoch 1/100
8/8 [==============================] - 0s 16ms/step - loss: 914.4391 - accuracy: 0.1133
Epoch 2/100
8/8 [==============================] - 0s 16ms/step - loss: 374.5955 - accuracy: 0.1055
Epoch 3/100
8/8 [==============================] - 0s 16ms/step - loss: 79.0396 - accuracy: 0.0898
Epoch 4/100
8/8 [==============================] - 0s 15ms/step - loss: 26.5781 - accuracy: 0.1211
Epoch 5/100
8/8 [==============================] - 0s 16ms/step - loss: 7.8244 - accuracy: 0.1016
Epoch 6/100
8/8 [==============================] - 0s 16ms/step - loss: 3.7817 - accuracy: 0.1211
Epoch 7/100
8/8 [==============================] - 0s 15ms/step - loss: 2.5276 - accuracy: 0.1016
Epoch 8/100
8/8 [==============================] - 0s 16ms/step - loss: 2.4878 - accuracy: 0.1055
Epoch 9/100
8/8 [==============================] - 0s 15ms/step - loss: 2.3278 - accuracy: 0.1016
Epoch 10/100
8/8 [==============================] - 0s 16ms/step - loss: 2.4341 - accuracy: 0.1016
Epoch 11/100


In [30]:
model_2.evaluate(X_test, y_test)


2/2 [==============================] - 0s 4ms/step - loss: 2.2857 - accuracy: 0.1250


[2.285698413848877, 0.125]

## Model 3

In [31]:
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define the input shape for your images
input_shape = X_train.shape[1:]

# Define a Sequential model or Functional API model
input_layer = Input(shape=input_shape)

# Add convolutional layers with pooling
conv1 = Conv2D(32, (3, 3), activation='relu')(input_layer)
pool1 = MaxPooling2D((2, 2))(conv1)

conv2 = Conv2D(64, (3, 3), activation='relu')(pool1)
pool2 = MaxPooling2D((2, 2))(conv2)

# Flatten the output before feeding into dense layers
flatten = Flatten()(pool2)

# Add dense layers with dropout for regularization
dense1 = Dense(128, activation='relu')(flatten)
dropout1 = Dropout(0.5)(dense1)  # Dropout layer for regularization

dense2 = Dense(64, activation='relu')(dropout1)
dropout2 = Dropout(0.5)(dense2)  # Dropout layer for regularization

# Output layer with softmax activation for multi-class classification
output_layer = Dense(y_train.shape[1], activation='softmax')(dropout2)

# Create the model
model_3 = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model_3.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Create an ImageDataGenerator with multiple augmentation techniques
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.5, 1.5],  # Vary brightness
    channel_shift_range=50,        # Vary channel shifts
    fill_mode='nearest'
)

# Fit the ImageDataGenerator to your data
datagen.fit(X_train)

# Example of using the ImageDataGenerator during model training
model_3.fit(datagen.flow(X_train, y_train, batch_size=32), epochs=100)


Epoch 1/100
8/8 [==============================] - 0s 23ms/step - loss: 53.5368 - accuracy: 0.0703
Epoch 2/100
8/8 [==============================] - 0s 26ms/step - loss: 2.8196 - accuracy: 0.1133
Epoch 3/100
8/8 [==============================] - 0s 27ms/step - loss: 2.3288 - accuracy: 0.0938
Epoch 4/100
8/8 [==============================] - 0s 24ms/step - loss: 2.3195 - accuracy: 0.1094
Epoch 5/100
8/8 [==============================] - 0s 26ms/step - loss: 2.3070 - accuracy: 0.0898
Epoch 6/100
8/8 [==============================] - 0s 23ms/step - loss: 2.2924 - accuracy: 0.1289
Epoch 7/100
8/8 [==============================] - 0s 24ms/step - loss: 2.2902 - accuracy: 0.1172
Epoch 8/100
8/8 [==============================] - 0s 22ms/step - loss: 2.3283 - accuracy: 0.1328
Epoch 9/100
8/8 [==============================] - 0s 22ms/step - loss: 2.3006 - accuracy: 0.1250
Epoch 10/100
8/8 [==============================] - 0s 23ms/step - loss: 2.2691 - accuracy: 0.1562
Epoch 11/100
8/8 [

In [32]:
model_3.evaluate(X_test, y_test)


2/2 [==============================] - 0s 6ms/step - loss: 2.2692 - accuracy: 0.1406


[2.269165515899658, 0.140625]

## Model 4

In [33]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

model_4 = Sequential()
model_4.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model_4.add(MaxPooling2D((2, 2)))
model_4.add(Conv2D(64, (3, 3), activation='relu'))
model_4.add(MaxPooling2D((2, 2)))
model_4.add(Conv2D(128, (3, 3), activation='relu'))
model_4.add(MaxPooling2D((2, 2)))
model_4.add(Flatten())
model_4.add(Dense(256, activation='relu', kernel_regularizer='l2'))  # Adding L2 regularization
model_4.add(Dropout(0.5))  # Adding Dropout layer
model_4.add(Dense(128, activation='relu', kernel_regularizer='l2'))  # Adding L2 regularization
model_4.add(Dense(y_train.shape[1], activation='softmax'))

# Using Adam optimizer with a reduced learning rate
optimizer = Adam(lr=0.0001)
model_4.compile(optimizer="adam",
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Adding more callbacks, e.g., ReduceLROnPlateau
callbacks = [EarlyStopping(patience=5, restore_best_weights=True),
             ReduceLROnPlateau(factor=0.1, patience=3)]

# Fit the model with increased complexity and callbacks
history = model_4.fit(datagen.flow(X_train, y_train, batch_size=16),
                    epochs=100,
                    callbacks=callbacks,
                    validation_data=(X_test, y_test))


Epoch 1/100


/Users/yassinebouaine/.pyenv/versions/3.10.6/envs/dynamic-players-insights/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


16/16 [==============================] - 1s 21ms/step - loss: 18.0604 - accuracy: 0.0781 - val_loss: 7.2218 - val_accuracy: 0.1406 - lr: 0.0010
Epoch 2/100
16/16 [==============================] - 0s 16ms/step - loss: 6.9503 - accuracy: 0.1094 - val_loss: 6.7495 - val_accuracy: 0.0312 - lr: 0.0010
Epoch 3/100
16/16 [==============================] - 0s 15ms/step - loss: 6.6170 - accuracy: 0.1367 - val_loss: 6.4980 - val_accuracy: 0.1094 - lr: 0.0010
Epoch 4/100
16/16 [==============================] - 0s 15ms/step - loss: 6.4001 - accuracy: 0.1406 - val_loss: 6.3007 - val_accuracy: 0.1250 - lr: 0.0010
Epoch 5/100
16/16 [==============================] - 0s 15ms/step - loss: 6.2040 - accuracy: 0.1211 - val_loss: 6.1111 - val_accuracy: 0.1406 - lr: 0.0010
Epoch 6/100
16/16 [==============================] - 0s 16ms/step - loss: 6.0245 - accuracy: 0.1445 - val_loss: 5.9336 - val_accuracy: 0.1250 - lr: 0.0010
Epoch 7/100
16/16 [==============================] - 0s 18ms/step - loss: 5.8554 

In [34]:
model_4.evaluate(X_test, y_test)


2/2 [==============================] - 0s 9ms/step - loss: 2.7581 - accuracy: 0.1719


[2.7581448554992676, 0.171875]

## Model 5

In [35]:
from keras.layers import BatchNormalization, LeakyReLU

model_5 = Sequential()
model_5.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model_5.add(MaxPooling2D((2, 2)))
model_5.add(BatchNormalization())  # Add BatchNormalization after each Conv layer
model_5.add(Conv2D(64, (3, 3), activation='relu'))
model_5.add(MaxPooling2D((2, 2)))
model_5.add(BatchNormalization())
model_5.add(Conv2D(128, (3, 3), activation='relu'))
model_5.add(MaxPooling2D((2, 2)))
model_5.add(BatchNormalization())
model_5.add(Flatten())
model_5.add(Dense(256))
model_5.add(LeakyReLU(alpha=0.1))  # LeakyReLU activation
model_5.add(Dropout(0.5))
model_5.add(Dense(128))
model_5.add(LeakyReLU(alpha=0.1))
model_5.add(Dense(y_train.shape[1], activation='softmax'))

# Compile the model
model_5.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Training the model with adjusted architecture
es = EarlyStopping(patience=10, restore_best_weights=True)
history = model_5.fit(datagen.flow(X_train, y_train, batch_size=64),
                    epochs=150,  # Train for more epochs
                    callbacks=es,
                    validation_data=(X_test, y_test))


Epoch 1/150
4/4 [==============================] - 1s 87ms/step - loss: 3.2009 - accuracy: 0.1211 - val_loss: 11.4369 - val_accuracy: 0.1562
Epoch 2/150
4/4 [==============================] - 0s 59ms/step - loss: 3.4948 - accuracy: 0.1250 - val_loss: 7.3437 - val_accuracy: 0.0938
Epoch 3/150
4/4 [==============================] - 0s 55ms/step - loss: 3.2415 - accuracy: 0.1367 - val_loss: 6.2547 - val_accuracy: 0.1094
Epoch 4/150
4/4 [==============================] - 0s 56ms/step - loss: 3.0423 - accuracy: 0.1562 - val_loss: 5.6266 - val_accuracy: 0.1094
Epoch 5/150
4/4 [==============================] - 0s 58ms/step - loss: 2.8201 - accuracy: 0.1992 - val_loss: 4.9246 - val_accuracy: 0.1250
Epoch 6/150
4/4 [==============================] - 0s 64ms/step - loss: 2.9792 - accuracy: 0.2148 - val_loss: 4.5927 - val_accuracy: 0.0625
Epoch 7/150
4/4 [==============================] - 0s 61ms/step - loss: 2.7867 - accuracy: 0.1523 - val_loss: 4.1531 - val_accuracy: 0.1250
Epoch 8/150
4/4 [==

In [36]:
model_5.evaluate(X_test, y_test)


2/2 [==============================] - 0s 9ms/step - loss: 1.7022 - accuracy: 0.3438


[1.7022076845169067, 0.34375]

# Video + Getty - 5 Classes (video_cropped_faces_2)

In [37]:
cropped_faces_directory = "../data_processing/raw_data/video_cropped_faces_2"

y, X = encode(cropped_faces_directory)
y_cat = to_cat(y)

X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.2)


## Model 1

In [38]:
# Add a new classification head
layer = resnet.output
layer = GlobalAveragePooling2D()(layer)
predictions = Dense(y_train.shape[1], activation='softmax')(layer)

model_1 = Model(inputs=resnet.input, outputs=predictions)

# Compile the model
model_1.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Callbacks
es = EarlyStopping(patience=3,
                   restore_best_weights=True)

# Train the model
model_1.fit(X_train,
          y_train,
          epochs=10,
          batch_size=16,
          validation_split=0.2,
          callbacks=[es]
         )

model_1.evaluate(X_test, y_test)


Epoch 1/10
8/8 [==============================] - 5s 283ms/step - loss: 1.3356 - accuracy: 0.5603 - val_loss: 249.2358 - val_accuracy: 0.1724
Epoch 2/10
8/8 [==============================] - 1s 171ms/step - loss: 0.8585 - accuracy: 0.8190 - val_loss: 807.2657 - val_accuracy: 0.3103
Epoch 3/10
8/8 [==============================] - 1s 171ms/step - loss: 0.5429 - accuracy: 0.8190 - val_loss: 1742.6189 - val_accuracy: 0.2759
Epoch 4/10
2/2 [==============================] - 0s 26ms/step - loss: 187.1584 - accuracy: 0.2432


[187.15835571289062, 0.2432432472705841]

## Model 2

In [39]:
# Define the input shape for your images
input_shape = X_train.shape[1:]

# Define a Sequential model or Functional API model
input_layer = Input(shape=input_shape)

# Add augmentation layers directly within the model
augmented = Conv2D(32, (3, 3), activation='relu')(input_layer)
augmented = MaxPooling2D((2, 2))(augmented)
# Add more layers or augmentation techniques as needed

# Rest of your model architecture
# Example:
augmented = Flatten()(augmented)
augmented = Dense(128, activation='relu')(augmented)
output_layer = Dense(y_train.shape[1], activation='softmax')(augmented)


model_2 = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model_2.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Callbacks
es = EarlyStopping(patience=3,
                   restore_best_weights=True)

datagen = ImageDataGenerator(
    rotation_range=40,       # Rotate images by a wider range of degrees
    width_shift_range=0.3,   # Shift images horizontally by a larger fraction of total width
    height_shift_range=0.3,  # Shift images vertically by a larger fraction of total height
    shear_range=0.3,         # Apply a larger shear-based transformation
    zoom_range=0.3,          # Zoom in or out of images more aggressively
    horizontal_flip=True,    # Flip images horizontally
    vertical_flip=True,      # Flip images vertically
    fill_mode='nearest'      # Fill points outside the boundaries using the nearest available value
)

# Fit the ImageDataGenerator to your data
datagen.fit(X_train)

# Example of using the ImageDataGenerator during model training
model_2.fit(datagen.flow(X_train, y_train, batch_size=32), epochs=100)

model_2.evaluate(X_test, y_test)


Epoch 1/100
5/5 [==============================] - 0s 14ms/step - loss: 1219.3690 - accuracy: 0.2276
Epoch 2/100
5/5 [==============================] - 0s 16ms/step - loss: 487.6217 - accuracy: 0.2000
Epoch 3/100
5/5 [==============================] - 0s 14ms/step - loss: 214.3429 - accuracy: 0.1931
Epoch 4/100
5/5 [==============================] - 0s 14ms/step - loss: 113.7920 - accuracy: 0.1724
Epoch 5/100
5/5 [==============================] - 0s 14ms/step - loss: 62.5287 - accuracy: 0.1862
Epoch 6/100
5/5 [==============================] - 0s 16ms/step - loss: 17.1502 - accuracy: 0.2552
Epoch 7/100
5/5 [==============================] - 0s 21ms/step - loss: 13.6125 - accuracy: 0.2966
Epoch 8/100
5/5 [==============================] - 0s 16ms/step - loss: 12.2949 - accuracy: 0.2621
Epoch 9/100
5/5 [==============================] - 0s 16ms/step - loss: 9.6691 - accuracy: 0.2897
Epoch 10/100
5/5 [==============================] - 0s 16ms/step - loss: 7.8384 - accuracy: 0.3310
Epoch 

[1.5021618604660034, 0.4864864945411682]

## Model 3

In [40]:
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define the input shape for your images
input_shape = X_train.shape[1:]

# Define a Sequential model or Functional API model
input_layer = Input(shape=input_shape)

# Add convolutional layers with pooling
conv1 = Conv2D(32, (3, 3), activation='relu')(input_layer)
pool1 = MaxPooling2D((2, 2))(conv1)

conv2 = Conv2D(64, (3, 3), activation='relu')(pool1)
pool2 = MaxPooling2D((2, 2))(conv2)

# Flatten the output before feeding into dense layers
flatten = Flatten()(pool2)

# Add dense layers with dropout for regularization
dense1 = Dense(128, activation='relu')(flatten)
dropout1 = Dropout(0.5)(dense1)  # Dropout layer for regularization

dense2 = Dense(64, activation='relu')(dropout1)
dropout2 = Dropout(0.5)(dense2)  # Dropout layer for regularization

# Output layer with softmax activation for multi-class classification
output_layer = Dense(y_train.shape[1], activation='softmax')(dropout2)

# Create the model
model_3 = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model_3.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Create an ImageDataGenerator with multiple augmentation techniques
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.5, 1.5],  # Vary brightness
    channel_shift_range=50,        # Vary channel shifts
    fill_mode='nearest'
)

# Fit the ImageDataGenerator to your data
datagen.fit(X_train)

# Example of using the ImageDataGenerator during model training
model_3.fit(datagen.flow(X_train, y_train, batch_size=32), epochs=100)

model_3.evaluate(X_test, y_test)


Epoch 1/100
5/5 [==============================] - 0s 22ms/step - loss: 71.9034 - accuracy: 0.1724
Epoch 2/100
5/5 [==============================] - 0s 25ms/step - loss: 12.2521 - accuracy: 0.1724
Epoch 3/100
5/5 [==============================] - 0s 22ms/step - loss: 2.4456 - accuracy: 0.1793
Epoch 4/100
5/5 [==============================] - 0s 22ms/step - loss: 1.9234 - accuracy: 0.1586
Epoch 5/100
5/5 [==============================] - 0s 21ms/step - loss: 1.7385 - accuracy: 0.2414
Epoch 6/100
5/5 [==============================] - 0s 22ms/step - loss: 1.6558 - accuracy: 0.2069
Epoch 7/100
5/5 [==============================] - 0s 23ms/step - loss: 1.6352 - accuracy: 0.2345
Epoch 8/100
5/5 [==============================] - 0s 21ms/step - loss: 1.5772 - accuracy: 0.2483
Epoch 9/100
5/5 [==============================] - 0s 20ms/step - loss: 1.6146 - accuracy: 0.1862
Epoch 10/100
5/5 [==============================] - 0s 20ms/step - loss: 1.6544 - accuracy: 0.2276
Epoch 11/100
5/5 

[1.5880852937698364, 0.3243243098258972]

## Model 4

In [41]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

model_4 = Sequential()
model_4.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model_4.add(MaxPooling2D((2, 2)))
model_4.add(Conv2D(64, (3, 3), activation='relu'))
model_4.add(MaxPooling2D((2, 2)))
model_4.add(Conv2D(128, (3, 3), activation='relu'))
model_4.add(MaxPooling2D((2, 2)))
model_4.add(Flatten())
model_4.add(Dense(256, activation='relu', kernel_regularizer='l2'))  # Adding L2 regularization
model_4.add(Dropout(0.5))  # Adding Dropout layer
model_4.add(Dense(128, activation='relu', kernel_regularizer='l2'))  # Adding L2 regularization
model_4.add(Dense(y_train.shape[1], activation='softmax'))

# Using Adam optimizer with a reduced learning rate
optimizer = Adam(lr=0.0001)
model_4.compile(optimizer="adam",
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Adding more callbacks, e.g., ReduceLROnPlateau
callbacks = [EarlyStopping(patience=5, restore_best_weights=True),
             ReduceLROnPlateau(factor=0.1, patience=3)]

# Fit the model with increased complexity and callbacks
history = model_4.fit(datagen.flow(X_train, y_train, batch_size=16),
                    epochs=100,
                    callbacks=callbacks,
                    validation_data=(X_test, y_test))

model_4.evaluate(X_test, y_test)


Epoch 1/100


/Users/yassinebouaine/.pyenv/versions/3.10.6/envs/dynamic-players-insights/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


10/10 [==============================] - 1s 27ms/step - loss: 34.2202 - accuracy: 0.1310 - val_loss: 8.1036 - val_accuracy: 0.1622 - lr: 0.0010
Epoch 2/100
10/10 [==============================] - 0s 15ms/step - loss: 7.6123 - accuracy: 0.1793 - val_loss: 6.1818 - val_accuracy: 0.1892 - lr: 0.0010
Epoch 3/100
10/10 [==============================] - 0s 17ms/step - loss: 6.0844 - accuracy: 0.1931 - val_loss: 5.9404 - val_accuracy: 0.1622 - lr: 0.0010
Epoch 4/100
10/10 [==============================] - 0s 15ms/step - loss: 5.8836 - accuracy: 0.2483 - val_loss: 5.7774 - val_accuracy: 0.1622 - lr: 0.0010
Epoch 5/100
10/10 [==============================] - 0s 16ms/step - loss: 5.7184 - accuracy: 0.2138 - val_loss: 5.6521 - val_accuracy: 0.2432 - lr: 0.0010
Epoch 6/100
10/10 [==============================] - 0s 15ms/step - loss: 5.5926 - accuracy: 0.2345 - val_loss: 5.5279 - val_accuracy: 0.1622 - lr: 0.0010
Epoch 7/100
10/10 [==============================] - 0s 15ms/step - loss: 5.5350 

[4.561069965362549, 0.37837839126586914]

## Model 5

In [42]:
from keras.layers import BatchNormalization, LeakyReLU

model_5 = Sequential()
model_5.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model_5.add(MaxPooling2D((2, 2)))
model_5.add(BatchNormalization())  # Add BatchNormalization after each Conv layer
model_5.add(Conv2D(64, (3, 3), activation='relu'))
model_5.add(MaxPooling2D((2, 2)))
model_5.add(BatchNormalization())
model_5.add(Conv2D(128, (3, 3), activation='relu'))
model_5.add(MaxPooling2D((2, 2)))
model_5.add(BatchNormalization())
model_5.add(Flatten())
model_5.add(Dense(256))
model_5.add(LeakyReLU(alpha=0.1))  # LeakyReLU activation
model_5.add(Dropout(0.5))
model_5.add(Dense(128))
model_5.add(LeakyReLU(alpha=0.1))
model_5.add(Dense(y_train.shape[1], activation='softmax'))

# Compile the model
model_5.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Training the model with adjusted architecture
es = EarlyStopping(patience=10, restore_best_weights=True)
history = model_5.fit(datagen.flow(X_train, y_train, batch_size=64),
                    epochs=150,  # Train for more epochs
                    callbacks=es,
                    validation_data=(X_test, y_test))


model_5.evaluate(X_test, y_test)


Epoch 1/150
3/3 [==============================] - 1s 91ms/step - loss: 2.3551 - accuracy: 0.2207 - val_loss: 15.2924 - val_accuracy: 0.1892
Epoch 2/150
3/3 [==============================] - 0s 42ms/step - loss: 2.9239 - accuracy: 0.2621 - val_loss: 12.1130 - val_accuracy: 0.1892
Epoch 3/150
3/3 [==============================] - 0s 72ms/step - loss: 2.3497 - accuracy: 0.3379 - val_loss: 3.5174 - val_accuracy: 0.1892
Epoch 4/150
3/3 [==============================] - 0s 54ms/step - loss: 2.7979 - accuracy: 0.2414 - val_loss: 5.0240 - val_accuracy: 0.1892
Epoch 5/150
3/3 [==============================] - 0s 69ms/step - loss: 2.3812 - accuracy: 0.3034 - val_loss: 6.6793 - val_accuracy: 0.1622
Epoch 6/150
3/3 [==============================] - 0s 44ms/step - loss: 2.1652 - accuracy: 0.3448 - val_loss: 7.3753 - val_accuracy: 0.1622
Epoch 7/150
3/3 [==============================] - 0s 59ms/step - loss: 2.4691 - accuracy: 0.2207 - val_loss: 5.2448 - val_accuracy: 0.1622
Epoch 8/150
3/3 [=

[1.2647937536239624, 0.5675675868988037]